# Importación de librerías

In [1]:
from DatasetLoader import DatasetLoader
from MethodDeepGCNResNet import MethodDeepGCNResNet
from MethodDeepGATResNet import MethodDeepGATResNet
from MethodDeepLoopyResNet import MethodDeepLoopyResNet
from ResultSaving import ResultSaving
from SettingCV import SettingCV
from EvaluateAcc import EvaluateAcc
import numpy as np
import torch.nn as nn
import torch

/home/luis/anaconda3/envs/TFM/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Carga de datasets

Se cargan los datasets citados en el paper (_Cora_, _Citeseet_ y _Pubmed_), inicializando además el número de clases correspondiente para cada uno.

Ejecutaremos el dataset **Cora**, que está configurado por defecto en el código de ejemplo:

In [2]:
#---- 'cora' , 'citeseer', 'pubmed' ----

dataset_name = 'cora'

if dataset_name == 'cora':
    nclass = 7
    nfeature = 1433
elif dataset_name == 'citeseer':
    nclass = 6
    nfeature = 3703
elif dataset_name == 'pubmed':
    nclass = 3
    nfeature = 500

# Ejecución

Se testan cada uno de los modelos citados en el Paper con los distintos residuos implementados para distintas profundidades.

Ejecutaremos las siguientes celdas para comprobar los resultados.

## Graph Convolutional Network

In [ ]:
Para comprender cómo se implementan los residuos en GCN, accederemos a la clase Method

In [3]:
#---- Deep GCN ResNet baseline method ----
if True:
    for depth in [1, 2, 3, 4, 5, 6, 7]:
        for residual_type in ['naive', 'raw', 'graph_naive', 'graph_raw']:

            acc_test_list = []
            print('Method: GCN, dataset: ' + dataset_name + ', depth: ' + str(depth) + ', residual type: ' + residual_type)
            for iter in range(1):
                #---- parameter section -------------------------------
                lr = 0.01
                epoch = 1000
                weight_decay = 5e-4
                c = 0.1
                seed = iter
                dropout = 0.5
                nhid = 16
                #------------------------------------------------------

                #---- objection initialization setction ---------------
                print('Start')

                data_obj = DatasetLoader('', '')
                data_obj.dataset_source_folder_path = './data/' + dataset_name + '/'
                data_obj.c = c
                data_obj.method_type = 'GCN'

                method_obj = MethodDeepGCNResNet(nfeature, nhid, nclass, dropout, seed, depth)
                method_obj.lr = lr
                method_obj.epoch = epoch
                method_obj.residual_type = residual_type

                result_obj = ResultSaving('', '')
                result_obj.result_destination_folder_path = './result/GResNet/'
                result_obj.result_destination_file_name = 'DeepGCNResNet_' + dataset_name + '_' + residual_type + '_depth_' + str(depth) + '_iter_' + str(iter)

                setting_obj = SettingCV('', '')

                evaluate_obj = EvaluateAcc('', '')
                #------------------------------------------------------

                #---- running section ---------------------------------
                setting_obj.prepare(data_obj, method_obj, result_obj, evaluate_obj)
                acc_test = setting_obj.load_run_save_evaluate()
                print('Testing Acc: ', acc_test)
                acc_test_list.append(acc_test)
                print('*******************************')
                #------------------------------------------------------
            print(acc_test_list)
            print(np.mean(acc_test_list), np.std(acc_test_list))
            print('Finished')

Method: GCN, dataset: cora, depth: 1, residual type: naive
Start
Loading <class 'dataset.dataset'> dataset...
(5429, 2)
Epoch: 0001 loss_train: 1.9547 acc_train: 0.1643 loss_val: 1.9391 acc_val: 0.1767 loss_test: 1.9407 acc_test: 0.1870 time: 0.0689s
Epoch: 0051 loss_train: 1.3644 acc_train: 0.9643 loss_val: 1.6899 acc_val: 0.5833 loss_test: 1.7191 acc_test: 0.5140 time: 0.0147s
Epoch: 0101 loss_train: 1.1674 acc_train: 0.9857 loss_val: 1.6006 acc_val: 0.6200 loss_test: 1.6376 acc_test: 0.5730 time: 0.0114s
Epoch: 0151 loss_train: 1.0857 acc_train: 0.9857 loss_val: 1.5571 acc_val: 0.6233 loss_test: 1.5968 acc_test: 0.5860 time: 0.0116s
Epoch: 0201 loss_train: 1.0429 acc_train: 0.9857 loss_val: 1.5303 acc_val: 0.6300 loss_test: 1.5714 acc_test: 0.5990 time: 0.0130s
Epoch: 0251 loss_train: 1.0173 acc_train: 0.9857 loss_val: 1.5119 acc_val: 0.6100 loss_test: 1.5536 acc_test: 0.6110 time: 0.0102s
Epoch: 0301 loss_train: 1.0005 acc_train: 0.9857 loss_val: 1.4984 acc_val: 0.6200 loss_test: 1

## Graph Attention Network

In [ ]:
#---- Deep Sparse GAT baseline method ----
if True:
    for depth in [1, 2, 3, 4, 5, 6, 7]:
        for residual_type in ['naive', 'raw', 'graph_naive', 'graph_raw']:

            cuda_tag = False
            print('Method: GAT, dataset: ' + dataset_name + ', depth: ' + str(depth) + ', residual type: ' + residual_type)
            method_type = 'GAT'
            acc_test_list = []
            for iter in range(1):
                # ---- parameter section -------------------------------
                lr = 0.005
                epoch = 500
                weight_decay = 5e-4
                c = 0.1
                seed = iter
                dropout = 0.6
                nhid = 8
                nb_heads = 8
                alpha = 0.2
                patience = 100
                # ------------------------------------------------------

                # ---- objection initialization setction ---------------
                print('Start')

                data_obj = DatasetLoader('', '')
                data_obj.dataset_source_folder_path = './data/' + dataset_name + '/'
                data_obj.c = c
                data_obj.method_type = method_type

                method_obj = MethodDeepGATResNet(nfeature, nhid, nclass, dropout, seed, alpha, nb_heads, depth, cuda_tag)

                method_obj.lr = lr
                method_obj.epoch = epoch
                method_obj.weight_decay = weight_decay
                method_obj.residual_type = residual_type

                result_obj = ResultSaving('', '')
                result_obj.result_destination_folder_path = './result/GResNet/'
                result_obj.result_destination_file_name = 'DeepGATResNet_' + dataset_name + '_' + residual_type + '_depth_' + str(
                    depth) + '_iter_' + str(iter)

                setting_obj = SettingCV('', '')

                evaluate_obj = EvaluateAcc('', '')
                # ------------------------------------------------------

                # ---- running section ---------------------------------
                setting_obj.prepare(data_obj, method_obj, result_obj, evaluate_obj)
                acc_test = setting_obj.load_run_save_evaluate()
                print('Testing Acc: ', acc_test)
                acc_test_list.append(acc_test)
                print('*******************************')
                # ------------------------------------------------------
            print(acc_test_list)
            print(np.mean(acc_test_list), np.std(acc_test_list))
            print('Finished')

Method: GAT, dataset: cora, depth: 1, residual type: naive
Start
Loading <class 'dataset.dataset'> dataset...
(5429, 2)
False


/home/luis/anaconda3/envs/TFM/lib/python3.9/site-packages/torch/optim/adam.py:81: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)


Epoch: 0001 loss_train: 1.9381 acc_train: 0.1714 loss_val: 1.9445 acc_val: 0.1500 loss_test: 1.9450 acc_test: 0.1490 time: 1.1278s
Epoch: 0011 loss_train: 1.8477 acc_train: 0.4500 loss_val: 1.8952 acc_val: 0.4167 loss_test: 1.9047 acc_test: 0.3510 time: 1.1868s
Epoch: 0021 loss_train: 1.7717 acc_train: 0.6714 loss_val: 1.8531 acc_val: 0.5167 loss_test: 1.8702 acc_test: 0.4290 time: 1.1950s
Epoch: 0031 loss_train: 1.7280 acc_train: 0.7143 loss_val: 1.8167 acc_val: 0.5367 loss_test: 1.8405 acc_test: 0.4580 time: 1.0811s
Epoch: 0041 loss_train: 1.6260 acc_train: 0.7571 loss_val: 1.7854 acc_val: 0.5667 loss_test: 1.8142 acc_test: 0.4650 time: 1.1785s
Epoch: 0051 loss_train: 1.6090 acc_train: 0.7714 loss_val: 1.7577 acc_val: 0.5867 loss_test: 1.7906 acc_test: 0.4820 time: 1.0010s
Epoch: 0061 loss_train: 1.5437 acc_train: 0.8000 loss_val: 1.7311 acc_val: 0.5867 loss_test: 1.7681 acc_test: 0.4930 time: 1.0402s
Epoch: 0071 loss_train: 1.4864 acc_train: 0.8286 loss_val: 1.7055 acc_val: 0.6067 l

## Deep LoopyNet

In [ ]:
#---- Deep LoopyNet baseline method ----
if True:
    for depth in [1, 2, 3, 4, 5, 6, 7]:
        for residual_type in ['naive', 'raw', 'graph_naive', 'graph_raw']:
            cuda_tag = False

            print('Method: LoopyNet, dataset: ' + dataset_name + ', depth: ' + str(depth) + ', residual type: ' + residual_type)
            acc_test_list = []
            for iter in range(1):
                #---- parameter section -------------------------------
                lr = 0.01
                epoch = 1000
                weight_decay = 5e-4
                c = 0.1
                seed = iter
                dropout = 0.5
                nhid = 16
                #------------------------------------------------------

                #---- objection initialization setction ---------------
                print('Start')

                data_obj = DatasetLoader('', '')
                data_obj.dataset_source_folder_path = './data/' + dataset_name + '/'
                data_obj.c = c
                data_obj.method_type = 'LoopyNet'

                method_obj = MethodDeepLoopyResNet(nfeature, nhid, nclass, dropout, seed, depth, cuda_tag)
                method_obj.lr = lr
                method_obj.epoch = epoch
                method_obj.residual_type = residual_type

                result_obj = ResultSaving('', '')
                result_obj.result_destination_folder_path = './result/GResNet/'
                result_obj.result_destination_file_name = 'DeepLoopyNetResNet_' + dataset_name + '_' + residual_type+'_depth_' + str(depth) + '_iter_' + str(iter)

                setting_obj = SettingCV('', '')

                evaluate_obj = EvaluateAcc('', '')
                #------------------------------------------------------

                #---- running section ---------------------------------
                setting_obj.prepare(data_obj, method_obj, result_obj, evaluate_obj)
                acc_test = setting_obj.load_run_save_evaluate()
                print('Testing Acc: ', acc_test)
                acc_test_list.append(acc_test)
                print('*******************************')
                #------------------------------------------------------
            print(acc_test_list)
            print(np.mean(acc_test_list), np.std(acc_test_list))
            print('Finished')
